In [ ]:
import altair as alt
import polars as pl

from utils import add_normalization_col


In [ ]:
df = add_normalization_col(
    pl.concat(
        [
            pl.read_csv(
                r"Q:\Data\PeerAgencyDashboards\airports\sfo-annual-updated250313.csv",
                columns=["year", "enplanements"],
            ).with_columns(airport=pl.lit("SFO")),
            pl.read_csv(
                r"Q:\Data\PeerAgencyDashboards\airports\oak-annual-updated250313.csv",
                columns=["year", "enplanements"],
            ).with_columns(airport=pl.lit("OAK")),
            pl.read_csv(
                r"Q:\Data\PeerAgencyDashboards\airports\sjc-annual-updated250313.csv",
                columns=["year", "enplanements"],
            ).with_columns(airport=pl.lit("SJC")),
        ]
    ).filter(pl.col("year") > 2017),
    "year",
    "enplanements",
    "enplanements (% of 2019 value)",
)
df.write_csv("output/data/airport_enplanements.csv")

In [ ]:
chart = (
    alt.Chart(df.with_columns(date=pl.date(pl.col("year"), 1, 1)))
    .mark_line()
    .encode(
        x=alt.X("date", title="year"),
        y=alt.Y("enplanements", title="airport enplanements").axis(format="s"),
        color=alt.Color("airport").scale(
            domain=["SFO", "OAK", "SJC"],
            range=["#ff8d6d", "#d4d756", "#8cb8ca"],
        ),
        tooltip=[
            "year",
            "airport",
            "enplanements",
            alt.Tooltip("enplanements (% of 2019 value)", format=".0%"),
        ],
    )
)
chart.save("output/Links/airport_enplanements.png")
chart.properties(width="container").save("output/Links/airport_enplanements.html")
chart


In [ ]:
# For monthly data only:
# plot = (
#     df.group_by("Year", "Month")
#     .agg(pl.sum("Enplanements"))
#     .sort("Year", "Month")
#     .plot.line(
#         x="Month",
#         y="Enplanements",
#         by="Year",
#         title="SFO enplanements",
#     )
# )
# hvplot.save(plot, "output/Links/sfo_enplanements-year_on_year.html")
# hvplot.save(plot, "output/Links/sfo_enplanements-year_on_year.png")
# plot

In [ ]:
# For monthly data only:
# plot = (
#     df.with_columns(date=pl.date(pl.col("Year"), pl.col("Month"), 1))
#     .group_by("date")
#     .agg(pl.sum("Enplanements"))
#     .sort("date")
#     .plot.line(
#         x="date",
#         y="Enplanements",
#         title="SFO enplanements",
#     )
# )
# hvplot.save(plot, "output/Links/sfo_enplanements-monthly.html")
# hvplot.save(plot, "output/Links/sfo_enplanements-monthly.png")
# plot